<a href="https://colab.research.google.com/github/ortenburger/apoHackathon/blob/master/presAggregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U tables
import os, json
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir("gdrive/My Drive/Hackathon")

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.5.2)
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [11]:
df =  pd.read_hdf('kkh_locs.h5')
df.head()

,IK,Standortnummer,Name,Jahr,Strasse,Hausnummer,Postleitzahl,Ort,IK-SO,Location,Bundesland,lat,lng
260000066_00_2013,260000066,00,Hochgebirgsklinik Davos,2013,Herman-Burchard-Str.,1,CH-7265,Davos Wolfgang,260000066-00,"[{'address_components': [{'long_name': '1', 's...",GR,46.8291,9.85558
260100023_00_2013,260100023,00,Diakonissenkrankenhaus Flensburg,2013,Knuthstraße,1,24939,Flensburg,260100023-00,"[{'address_components': [{'long_name': '1', 's...",SH,54.7904,9.42605
260100034_00_2013,260100034,00,Malteser Krankenhaus St. Franziskus-Hospital,2013,Waldstraße,17,24939,Flensburg,260100034-00,"[{'address_components': [{'long_name': '17', '...",SH,54.7925,9.41987
260100089_00_2013,260100089,00,Städtisches Krankenhaus Kiel GmbH,2013,Chemnitzstr.,33,24116,Kiel,260100089-00,"[{'address_components': [{'long_name': '33', '...",SH,54.3231,10.1161
260100125_00_2013,260100125,00,Sankt Elisabeth Krankenhaus Kiel,2013,Königsweg,8,24103,Kiel,260100125-00,"[{'address_components': [{'long_name': '8', 's...",SH,54.3169,10.1272


In [0]:
newdf = df.set_index(['IK-SO', 'Jahr'], verify_integrity=False)

In [8]:
newdf.head()

,,IK,Standortnummer,Name,Strasse,Hausnummer,Postleitzahl,Ort,Location
IK-SO,Jahr,,,,,,,,
260000066-00,2013,260000066,00,Hochgebirgsklinik Davos,Herman-Burchard-Str.,1,CH-7265,Davos Wolfgang,"[{'address_components': [{'long_name': '1', 's..."
260100023-00,2013,260100023,00,Diakonissenkrankenhaus Flensburg,Knuthstraße,1,24939,Flensburg,"[{'address_components': [{'long_name': '1', 's..."
260100034-00,2013,260100034,00,Malteser Krankenhaus St. Franziskus-Hospital,Waldstraße,17,24939,Flensburg,"[{'address_components': [{'long_name': '17', '..."
260100089-00,2013,260100089,00,Städtisches Krankenhaus Kiel GmbH,Chemnitzstr.,33,24116,Kiel,"[{'address_components': [{'long_name': '33', '..."
260100125-00,2013,260100125,00,Sankt Elisabeth Krankenhaus Kiel,Königsweg,8,24103,Kiel,"[{'address_components': [{'long_name': '8', 's..."


In [13]:
aggDict=[]
for index, row in df.iterrows():
  aggDict.append({"IK-SO": row["IK-SO"], 'year': row['Jahr'], 'lng':row['lng'], 'lat':row['lat'], 'state':row['Bundesland']})
print(aggDict[100])

{'IK-SO': '260200273-00', 'year': '2013', 'lng': 10.1536299, 'lat': 53.75758, 'state': 'SH'}


In [19]:
tdf =  pd.read_hdf('df_traeger.h5')
tdf.head()
tdict = []
for row in aggDict:
  try:
    row['htype'] = tdf.loc[row['IK-SO'].split('-')[0]+'_'+row['year']]['Trägerart']
  except:
    row['htype'] = 'Ohne'
  tdict.append(row)
print(tdict[100])

{'IK-SO': '260200273-00', 'year': '2013', 'lng': 10.1536299, 'lat': 53.75758, 'state': 'SH', 'htype': 'freigemeinnützig'}


In [22]:
aggdf = pd.DataFrame.from_dict(tdict)
aggdf.head()

,IK-SO,htype,lat,lng,state,year
0,260000066-00,privat,46.829140,9.855576,GR,2013
1,260100023-00,freigemeinnützig,54.790410,9.426050,SH,2013
2,260100034-00,freigemeinnützig,54.792500,9.419870,SH,2013
3,260100089-00,öffentlich,54.323074,10.116095,SH,2013
4,260100125-00,freigemeinnützig,54.316870,10.127190,SH,2013


In [24]:
newdf = aggdf.set_index(['IK-SO', 'year'])
newdf.head()

,,htype,lat,lng,state
IK-SO,year,,,,
260000066-00,2013,privat,46.829140,9.855576,GR
260100023-00,2013,freigemeinnützig,54.790410,9.426050,SH
260100034-00,2013,freigemeinnützig,54.792500,9.419870,SH
260100089-00,2013,öffentlich,54.323074,10.116095,SH
260100125-00,2013,freigemeinnützig,54.316870,10.127190,SH


In [31]:
icddf =  pd.read_hdf('df_icdmap (1).h5')
icddf.head()


,Class,Chapter,ICD_10,Title
0,3,01,A00,Cholera
1,4,01,A000,"Cholera durch Vibrio cholerae O:1, Biovar chol..."
2,4,01,A001,"Cholera durch Vibrio cholerae O:1, Biovar eltor"
3,4,01,A009,"Cholera, nicht näher bezeichnet"
4,3,01,A01,Typhus abdominalis und Paratyphus


In [36]:
icdList=[]
for index, row in icddf.iterrows():
  icdList.append(row['Chapter'])
icdList = list(set(icdList))
icdList.sort()
print(icdList)

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22']


In [0]:
for i in icdList:
  newdf['ICD' + str(i)] = 0

In [38]:
newdf.head()

,,htype,lat,lng,state,ICD01,ICD02,ICD03,ICD04,ICD05,ICD06,ICD07,ICD08,ICD09,ICD10,ICD11,ICD12,ICD13,ICD14,ICD15,ICD16,ICD17,ICD18,ICD19,ICD20,ICD21,ICD22
IK-SO,year,,,,,,,,,,,,,,,,,,,,,,,,,,
260000066-00,2013,privat,46.829140,9.855576,GR,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
260100023-00,2013,freigemeinnützig,54.790410,9.426050,SH,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
260100034-00,2013,freigemeinnützig,54.792500,9.419870,SH,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
260100089-00,2013,öffentlich,54.323074,10.116095,SH,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
260100125-00,2013,freigemeinnützig,54.316870,10.127190,SH,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [40]:
newdf.to_hdf('aggdata.h5', key='kkhicd')

/usr/local/lib/python3.6/dist-packages/tables/attributeset.py:475: NaturalNameWarning: object name is not a valid Python identifier: 'axis1_nameIK-SO'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['htype', 'state']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [95]:
icddf =  pd.read_hdf('df_icd.h5')
icdmap = pd.read_hdf('df_icdmap (1).h5')
icddf['ICD'] = None
icddf.head()


,IK,Standortnummer,Jahr,ICD_10,Fallzahl,ICD
260000066_00_2013_0,260000066,00,2013,B44,17,None
260000066_00_2013_1,260000066,00,2013,D86,7,None
260000066_00_2013_2,260000066,00,2013,J12,0,None
260000066_00_2013_3,260000066,00,2013,J30,9,None
260000066_00_2013_4,260000066,00,2013,J44,19,None


In [103]:
#icdmap.head()
print(icdmap.loc[icdmap['ICD_10'] == 'B59']['Chapter'])
"""icdDict={}
for index, row in icdmap.iterrows():
  icdDict[row['ICD_10']]=row['Chapter']

print(icdDict)"""

Series([], Name: Chapter, dtype: int64)


"icdDict={}\nfor index, row in icdmap.iterrows():\n  icdDict[row['ICD_10']]=row['Chapter']\n\nprint(icdDict)"

In [0]:
foo=[]
i=0
for index, row in icddf.iterrows():
  #print(index)
  if not row['ICD']:
    try:
      icddf.loc[index, 'ICD'] = icdDict[row['ICD_10']]
    except:
      try:
        icddf.loc[index, 'ICD'] = icdDict[row['ICD_10'].split('.')[0]]
      except:
        pass

{'A00': 1, 'A000': 1, 'A001': 1, 'A009': 1, 'A01': 1, 'A010': 1, 'A011': 1, 'A012': 1, 'A013': 1, 'A014': 1, 'A02': 1, 'A020': 1, 'A021': 1, 'A022': 1, 'A028': 1, 'A029': 1, 'A03': 1, 'A030': 1, 'A031': 1, 'A032': 1, 'A033': 1, 'A038': 1, 'A039': 1, 'A04': 1, 'A040': 1, 'A041': 1, 'A042': 1, 'A043': 1, 'A044': 1, 'A045': 1, 'A046': 1, 'A047': 1, 'A0470': 1, 'A0471': 1, 'A0472': 1, 'A0473': 1, 'A0479': 1, 'A048': 1, 'A049': 1, 'A05': 1, 'A050': 1, 'A051': 1, 'A052': 1, 'A053': 1, 'A054': 1, 'A058': 1, 'A059': 1, 'A06': 1, 'A060': 1, 'A061': 1, 'A062': 1, 'A063': 1, 'A064': 1, 'A065': 1, 'A066': 1, 'A067': 1, 'A068': 1, 'A069': 1, 'A07': 1, 'A070': 1, 'A071': 1, 'A072': 1, 'A073': 1, 'A078': 1, 'A079': 1, 'A08': 1, 'A080': 1, 'A081': 1, 'A082': 1, 'A083': 1, 'A084': 1, 'A085': 1, 'A09': 1, 'A090': 1, 'A099': 1, 'A15': 1, 'A150': 1, 'A151': 1, 'A152': 1, 'A153': 1, 'A154': 1, 'A155': 1, 'A156': 1, 'A157': 1, 'A158': 1, 'A159': 1, 'A16': 1, 'A160': 1, 'A161': 1, 'A162': 1, 'A163': 1, 'A164